In [ ]:
import sys
sys.path.append("modules")

In [ ]:
import os, glob
from PIL import Image

import numpy as np
import scipy.interpolate as interp
import scipy.fft as fft
from scipy.stats import linregress

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import modules.time_to_H as time_to_H
import modules.compute_flexion as flex
import modules.parameters as setup

In [ ]:
# Read data

pathExp = "../data/testPolymer/withScotch"

pathFields = os.path.join(pathExp, "muDIC_3")
pathSnapshots = os.path.join(pathExp, "snapshots_preProcessed")

h_layer = 5e-3
dt_camera = 0.1

# For cropped data
xMin_crop, xMax_crop = 758, 1772
yMin_crop, yMax_crop = 523, 1165

frames_true = np.load(os.path.join(pathFields, "frames.npy")).astype(int)
coords = np.load(os.path.join(pathFields, "coords.npy"))
trueStrain_raw = np.load(os.path.join(pathFields, "true_strain.npy"))
trueStrainXX_raw = trueStrain_raw[0,0,0]

times_true = dt_camera * frames_true
heights = time_to_H.time_to_H(times_true, Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax)

with open(os.path.join(pathFields, 'params.txt')) as file:
    paramsDIC = file.read()

# Undo the change of origin due to cropping
coords[0,0] += xMin_crop
coords[0,1] += yMin_crop

In [ ]:
# Read calibration

pathCalibration = "../data/Calibrations/Top_grey_02_08/muDIC"

dt_camera_calib = 0.1

# For cropped data
xMin_crop_calib, xMax_crop_calib = 0, 2448
yMin_crop_calib, yMax_crop_calib = 0, 2048

frames_true_calib = np.load(os.path.join(pathCalibration, "frames.npy"))
coords_calib = np.load(os.path.join(pathCalibration, "coords.npy"))
trueStrain_calib = np.load(os.path.join(pathCalibration, "true_strain.npy"))

times_true_calib = dt_camera_calib * frames_true_calib
heights_calib = time_to_H.time_to_H(times_true_calib, Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax)

# Undo the change of origin due to cropping
coords_calib[0,0] += xMin_crop_calib
coords_calib[0,1] += yMin_crop_calib

# Keep only the rising part
duration_rising = time_to_H.compute_time_rising(Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax)
mask_rising_calib = (times_true_calib <= duration_rising)

frames_true_calib = frames_true_calib[mask_rising_calib]
coords_calib = coords_calib[:,:,:,:,mask_rising_calib]
trueStrain_calib = trueStrain_calib[:,:,:,:,:,mask_rising_calib]
times_true_calib = times_true_calib[mask_rising_calib]
heights_calib = heights_calib[mask_rising_calib]

In [ ]:
# Test position

xExtend = coords[0,0,:,:,0].min(), coords[0,0,:,:,0].max()
yExtend = coords[0,1,:,:,0].min(), coords[0,1,:,:,0].max()

xExtend_calib = coords_calib[0,0,:,:,0].min(), coords_calib[0,0,:,:,0].max()
yExtend_calib = coords_calib[0,1,:,:,0].min(), coords_calib[0,1,:,:,0].max()

fig, ax = plt.subplots()

ax.plot((xExtend[0], xExtend[1], xExtend[1], xExtend[0]), (yExtend[0], yExtend[0], yExtend[1], yExtend[1]), 'bo')
ax.plot((xExtend_calib[0], xExtend_calib[1], xExtend_calib[1], xExtend_calib[0]), 
        (yExtend_calib[0], yExtend_calib[0], yExtend_calib[1], yExtend_calib[1]), 'ko')

plt.show()

In [ ]:
# Print shapes

print("From experiment")
print(f"Frames: {frames_true.shape}")
print(f"Coordinates/Strain: {coords[0,0].shape}")

print("")

print("From calibration")
print(f"Frames: {frames_true_calib.shape}")
print(f"Coordinates/Strain: {coords_calib[0,0].shape}")

In [ ]:
# Prepare snapshots prefix

snapshot_names = glob.glob(pathSnapshots + "/*.tiff")

snapshot_prefix = snapshot_names[0].rsplit("_")
snapshot_prefix = "_".join(snapshot_prefix[:-1])

In [ ]:
## Fit center curcature with parabola

plotFit = True

x0 = coords[0,0,:,:,0]
x0_calib = coords_calib[0,0,:,:,0]

# For experiments
trueStrainXX_raw_avg_Y = trueStrainXX_raw.mean(axis=1)
x0_arr = x0[:,x0.shape[1]//2]

poly_coeff = np.polyfit(x0_arr, trueStrainXX_raw_avg_Y, deg=2)
x_maxStrain = np.where(np.abs(poly_coeff[0])>1e-15, -poly_coeff[1] / (2 * poly_coeff[0]), 0)

# For calibration
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1)
x0_arr_calib = x0_calib[:,x0_calib.shape[1]//2]

poly_coeff_calib = np.polyfit(x0_arr_calib, trueStrain_calib_xx, deg=2)
x_maxStrain_calib = np.where(np.abs(poly_coeff_calib[0])>1e-15, -poly_coeff_calib[1] / (2 * poly_coeff_calib[0]), 0)


if plotFit:
    frameIndex_toPlot = 90
    fig, ax = plt.subplots(ncols=2, figsize=(10,5))

    ax[0].plot(x0_arr, trueStrainXX_raw_avg_Y[:,frameIndex_toPlot], label="Measure")
    ax[0].plot(x0_arr, np.poly1d(poly_coeff[:,frameIndex_toPlot])(x0_arr), 'k:', label="Fit")
    ax[0].axvline(x_maxStrain[frameIndex_toPlot], ls="--", c='k')

    ax[0].legend()
    ax[0].set_title(f"Frame index {frameIndex_toPlot}")

    ax[1].plot(x_maxStrain)
    ax[1].axhline(x0_arr.mean(), c='k')
    ax[1].set_xlabel('Frame index')
    ax[1].set_ylabel('X_maxStrain')

    plt.show()

In [ ]:
# Create interpolator calibration (x0,t)->eps_t

# Average calibration along y
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1)
x0_calib = coords_calib[0,0,:,:,0].mean(axis=1)

# Create interpolator
trueStrain_calib_interp_fromX0 = interp.RegularGridInterpolator((x0_calib, times_true_calib), 
                                                         trueStrain_calib_xx, 
                                                         bounds_error=False, fill_value=None)


In [ ]:
# Create interpolator calibration (x(t),h)->eps_t

# Average calibration along y
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1) # shape (x,time)
x_calib = coords_calib[0,0,:,:,:].mean(axis=1)                   # shape (x,time)
heights_calib_2d = np.array([heights_calib for _ in range(x_calib.shape[0])])

# Create interpolator
trueStrain_calib_interp_fromXt = interp.LinearNDInterpolator(list(zip(x_calib.flatten(), heights_calib_2d.flatten())), 
                                                      trueStrain_calib_xx.flatten(),
                                                      fill_value=np.nan, rescale=True)

# Prepare the data (using calibration)

In [ ]:
# Correct the strain

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

trueStrainXX_correction = np.empty_like(trueStrain_raw[0,0,0])
curvatures = np.empty((2, times_true.size))

for i_time in range(times_true.size):

    time = times_true[i_time]
    height = heights[i_time]

    # Compute correction from (x,t)
    trueStrainXX_correction[:,:,i_time] = trueStrain_calib_interp_fromXt((coords[0,0,:,:,i_time].flatten(), 
                                                                          np.full(x0.size, height))).reshape(x0.shape)
    
    # Compute correction from (X0,t)
    #trueStrainXX_correction[:,:,i_time] = trueStrain_calib_interp_fromX0((x0.flatten(), 
    #                                                                      np.full(x0.size, time))).reshape(x0.shape)
    
    guess_alpha = [0.1, 0.01]
    curvatures[:,i_time] = flex.solve_curvature(setup.W, heights[i_time], setup.a, guess_alpha)

trueStrainXX_corrected = trueStrainXX_raw - trueStrainXX_correction
engStrainXX_corrected = np.exp(trueStrainXX_corrected) - 1

# At fixed frame

In [ ]:
# Compare contour true_strain without and with calibration

frame_index = 100

frame_true = frames_true[frame_index]
time_true = times_true[frame_index]
height = heights[frame_index]

print(f'Time: {time_true}s')

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

trueStrain_corrected_toPlot = trueStrainXX_corrected[:,:,frame_index]
trueStrain_raw_toPlot = trueStrainXX_raw[:,:,frame_index]

fig, ax = plt.subplots(figsize=(15,5), ncols=2)

ax[0].set_title(f"True strain raw [frame {frame_true}]")
ax[1].set_title(f"True strain corrected [frame {frame_true}]")

im = ax[0].contourf(x0, y0, trueStrain_raw_toPlot)
fig.colorbar(im, ax=ax[0], label="True strain $\epsilon^t_{xx}$ [-]")

im = ax[1].contourf(x0, y0, trueStrain_corrected_toPlot)
fig.colorbar(im, ax=ax[1], label="True strain $\epsilon^t_{xx}$ [-]")

for axi in ax:
    axi.set_xlabel('$X_0$ position [pix]')
    axi.set_ylabel('$Y_0$ position [pix]')

plt.show()

In [ ]:
# Plot the strain along given lines

frame_index = 40
offset_H = 0e-3

y_pix_list = [600, 700, 800, 900, 1000]
#y_pix_list = [1400, 1500, 1600, 1700]

print(f"Min/Max y-coord: {y0.min(), y0.max()}")

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y0[y0.shape[0]//2,:]))
    y_elem_list.append(y_elem)

# Plot
fig, ax = plt.subplots(figsize=(10, 5), ncols=2, tight_layout=True)

for y_pix, y_elem in zip(y_pix_list, y_elem_list):
    ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,y_elem,frame_index], label=f'y_pix = {y_pix}')
    ax[1].plot(x0[:,y_elem], trueStrainXX_corrected[:,y_elem,frame_index], label=f'y_pix = {y_pix}')

# Plot the y-averaged
ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,:,frame_index].mean(axis=1), label=f'y-averaged', c='k')
ax[1].plot(x0[:,y_elem], trueStrainXX_corrected[:,:,frame_index].mean(axis=1), label=f'y-averaged', c='k')


# Plot the theoretical strain due to flexion
kappa_min, kappa_max = flex.solve_curvature(setup.W, heights[frame_index]-offset_H, setup.a)
ax[1].axhline(np.log(1 + kappa_max * h_layer), ls=':', c='k', label="$\log \\left( 1 + \\kappa h \\right)$")

for axi in ax:
    axi.set_xlabel('Horizontal position $X_0$ [pix]')
    axi.legend(loc='best', ncol=2)

ax[0].set_ylabel("True strain $\epsilon^t_{xx}$ [-]")
ax[1].set_ylabel("True strain $\epsilon^t_{xx} - \epsilon^t_{xx, calib}$ [-]")

ax[0].set_title(f"True strain [H = {heights[frame_index] * 1e3:.1f}mm]")
ax[1].set_title(f"True strain (corrected) [H = {heights[frame_index] * 1e3:.1f}mm]")


plt.show()

# Generate videos

In [ ]:
# Plot trueStrain corrected only

nbFrames = coords.shape[-1]
frameIndex_start = 100

fig, ax = plt.subplots()
fig.set_dpi(200)

# Set labels
ax.set_title(f"True strain [H = {1e3 * heights[frameIndex_start]:.1f}mm]")
ax.set_xlabel('Position $X_0$ [pix]')
ax.set_ylabel('Position $Y_0$ [pix]')

# Set ref coordinates and strain
x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

# Set bounds
xbounds = x0.min(), x0.max()
ybounds = y0.min(), y0.max()
ax.set_xbound(*xbounds)
ax.set_ybound(*ybounds)

# Set static cmap
vmin, vmax = np.nanmin(trueStrainXX_corrected), np.nanmax(trueStrainXX_corrected)
norm = plt.Normalize(vmin, vmax)
cmap = plt.get_cmap('viridis')
n_levels = 20

# Plot strating frame
trueStrain_toPlot = trueStrainXX_corrected[:,:,frameIndex_start]

cont = ax.contourf(x0, y0, trueStrain_toPlot, levels=n_levels, cmap=cmap, norm=norm)
fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, label="True strain $\epsilon^t_{xx}$ [-]")


def update(frame_index):
    global cont

    trueStrain_toPlot = trueStrainXX_corrected[:,:,frame_index]
    
    for c in cont.collections:
        c.remove()  # removes only the contours, leaves the rest intact

    cont = plt.contourf(x0, y0, trueStrain_toPlot, levels=n_levels, cmap=cmap, norm=norm)

    # Set limits and title
    ax.set_title(f"True strain [H = {1e3 * heights[frame_index]:.1f}mm]")
    ax.set_xbound(*xbounds)
    ax.set_ybound(*ybounds)
    
    return cont

anim = animation.FuncAnimation(fig, update, frames=nbFrames, repeat=False, interval=100)
anim_html = HTML(anim.to_html5_video())

print(f"{nbFrames} frames")


In [ ]:
# Display animation

anim_html

In [ ]:
# Save animation

savepath = os.path.join(pathFields, "animation_trueStrain.html")
with open(savepath, 'w') as f:
    f.write(anim_html.data)

In [ ]:
def reshape_bounds(bounds, eps):
    gapSide = eps * (bounds[1] - bounds[0])
    return bounds[0] - gapSide, bounds[1] + gapSide

In [ ]:
# Plot the strain along given lines

frame_index_start = 50
nbFrames = trueStrainXX_raw.shape[-1]

y_pix_list = [600, 700, 800, 900, 1000]
#y_pix_list = [1400, 1500, 1600, 1700]

print(f"Min/Max y-coord: {y0.min(), y0.max()}")

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y0[y0.shape[0]//2,:]))
    y_elem_list.append(y_elem)
print(y_elem_list)

# Create figure
fig, ax = plt.subplots(figsize=(10, 5), ncols=2, tight_layout=True)
fig.set_dpi(200)

# Set titles
ax[0].set_title(f"True strain (raw) [H = {heights[frame_index_start] * 1e3:.1f}mm]")
ax[1].set_title(f"True strain (corrected) [H = {heights[frame_index_start] * 1e3:.1f}mm]")

# Set labels
for axi in ax:
    axi.set_xlabel('$X_0$ position [pix]')
    axi.set_ylabel("True strain $\epsilon^t_{xx}$ [-]")

# Set bounds
xbounds_all = coords[0,0,:,0,0].min(), coords[0,0,:,0,0].max()
ybounds_raw = trueStrainXX_raw[:,y_elem_list,:].min(), trueStrainXX_raw[:,y_elem_list,:].max()
ybounds_corr = trueStrainXX_corrected[:,y_elem_list,:].min(), trueStrainXX_corrected[:,y_elem_list,:].max()

xbounds_all = reshape_bounds(xbounds_all, 0.02)
ybounds_raw = reshape_bounds(ybounds_raw, 0.02)
ybounds_corr = reshape_bounds(ybounds_corr, 0.02)
ax[0].set_xbound(*xbounds_all); ax[1].set_xbound(*xbounds_all)
ax[0].set_ybound(*ybounds_raw); ax[1].set_ybound(*ybounds_corr)

# Plot initial frame
lines_raw = [ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,y_elem,frame_index_start], label=f"Y = {y_pix}pix") 
             for y_elem, y_pix in zip(y_elem_list, y_pix_list)]
lines_corr = [ax[1].plot(x0[:,y_elem], trueStrainXX_corrected[:,y_elem,frame_index_start], label=f"Y = {y_pix}pix") 
              for y_elem, y_pix in zip(y_elem_list, y_pix_list)]

kappa_min, kappa_max = flex.solve_curvature(setup.W, heights[frame_index_start], setup.a)
line_th = ax[1].axhline(np.log(1 + kappa_max * h_layer), ls=':', c='k', label="$\log \\left( 1 + \\kappa h \\right)$")
lines_yAvg = [ax[0].plot(x0.mean(axis=1), trueStrainXX_raw[:,:,frame_index_start].mean(axis=1), label=f'y-averaged', c='k'),
              ax[1].plot(x0.mean(axis=1), trueStrainXX_corrected[:,:,frame_index_start].mean(axis=1), label=f'y-averaged', c='k')]

# Set legend
ax[0].legend(loc='upper center', bbox_to_anchor=(0.5, -.15), ncol=3, fancybox=True, shadow=True)
ax[1].legend(loc='upper center', bbox_to_anchor=(0.5, -.15), ncol=3, fancybox=True, shadow=True)
#fig.legend([line[0] for line in lines_corr[:2]], ('Line 1', 'Line 2'), 'lower center')
#ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

def update(frame_index):
    global lines_raw, lines_corr, line_th, lines_yAvg

    # Plot lines (x0, eps_t)
    for line, y_elem in zip(lines_raw, y_elem_list):
        line[0].set_ydata(trueStrainXX_raw[:,y_elem,frame_index])
    for  line, y_elem in zip(lines_corr, y_elem_list):
        line[0].set_ydata(trueStrainXX_corrected[:,y_elem,frame_index])

    # Plot yAvg data
    lines_yAvg[0][0].set_ydata(trueStrainXX_raw[:,:,frame_index].mean(axis=1))
    lines_yAvg[1][0].set_ydata(trueStrainXX_corrected[:,:,frame_index].mean(axis=1))

    # Plot theoretical prediction
    _, kappa_max = flex.solve_curvature(setup.W, heights[frame_index], setup.a)
    trueStrain_theory = np.log(1 + kappa_max * h_layer)
    line_th.set_data([0,1], [trueStrain_theory, trueStrain_theory])

    # Set limits and title
    ax[0].set_title(f"True strain (raw) [H = {heights[frame_index] * 1e3:.1f}mm]")
    ax[1].set_title(f"True strain (corrected) [H = {heights[frame_index] * 1e3:.1f}mm]")
    ax[0].set_xbound(*xbounds_all); ax[1].set_xbound(*xbounds_all)
    ax[0].set_ybound(*ybounds_raw); ax[1].set_ybound(*ybounds_corr)

    return lines_raw, lines_corr, line_th, lines_yAvg

anim = animation.FuncAnimation(fig, update, frames=nbFrames, repeat=False, interval=100)
anim_html = HTML(anim.to_html5_video())


In [ ]:
anim_html

# Time-dependant statistics

## Average and deviation strain

In [ ]:
# Mask for stats

# Limits to modify (in pix)
xMin, xMax = 800, 1600
yMin, yMax = int(y0.min()), int(y0.max())

# Compute the equivalent elements
# TODO: maskXY

In [ ]:
# Plot the evolution of the mean true strain as a function of time and maximal curvature

spatialMean_trueStrain_corrected = trueStrainXX_corrected.mean(axis=(0,1))

fig, ax = plt.subplots(figsize=(10,4), ncols=2, tight_layout=True)
ax[0].set_title("True strain average as function of elevation")
ax[1].set_title("True strain average as function of curvature")

# First plot: elevation
ax[0].plot(heights, spatialMean_trueStrain_corrected, label="$\\langle \epsilon^t_{xx} \\rangle_{xy}$")
ax[0].plot(heights, np.log(1 + h_layer * curvatures[1]), ls=':', c='k', label="$\log(1 + \kappa h)$")

# Second plot: curvature
ax[1].plot(curvatures[1], spatialMean_trueStrain_corrected, label="$\\langle \epsilon^t_{xx} \\rangle_{xy}$")
ax[1].plot(curvatures[1], np.log(1 + h_layer * curvatures[1]), ls=':', c='k', label="$\log(1 + \kappa h)$")


ax[0].set_xlabel("Cylinder height H [m]")
ax[1].set_xlabel("Central curvature $\\kappa_{max}$ [$m^{-1}$]")
for axi in ax:
    axi.set_ylabel("True strain $\\langle \epsilon^t_{xx} \\rangle_{xy}$ [-]")
    axi.legend()

plt.show()

In [ ]:
# Plot the evolution of the mean engineering strain as a function of time and maximal curvature

spatialMean_engStrain_corrected = engStrainXX_corrected.mean(axis=(0,1))
mask_up = (times_true < time_to_H.compute_time_rising(Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax))

# Compute the linear regressions
kMax_bounds_1 = [1, 4]
#kMax_bounds_2 = [6, 8]
mask_bounds_1 = (kMax_bounds_1[0] < curvatures[1]) & (curvatures[1] < kMax_bounds_1[1])
#mask_bounds_2 = (kMax_bounds_2[0] < curvatures[1]) & (curvatures[1] < kMax_bounds_2[1])

regress_1 = linregress(curvatures[1,mask_up&mask_bounds_1], spatialMean_engStrain_corrected[mask_up&mask_bounds_1])
#regress_2 = linregress(curvatures[1,mask_up&mask_bounds_2], spatialMean_engStrain_corrected[mask_up&mask_bounds_2])

fig, ax = plt.subplots(figsize=(10,4), ncols=2, tight_layout=True)
ax[0].set_title("Engineering strain average as function of elevation")
ax[1].set_title("Engineering strain average as function of curvature")

# First plot: elevation
ax[0].plot(heights[mask_up], spatialMean_engStrain_corrected[mask_up], label="$\\langle \epsilon_{xx} \\rangle_{xy}$")
ax[0].plot(heights[mask_up], h_layer * curvatures[1,mask_up], ls=':', c='k', label="$\kappa h$")

# Second plot: curvature
ax[1].plot(curvatures[1,mask_up], spatialMean_engStrain_corrected[mask_up], label="$\\langle \epsilon_{xx} \\rangle_{xy}$")
ax[1].plot(curvatures[1,mask_up], h_layer * curvatures[1,mask_up], ls='--', c='k', label="$\kappa h$")
ax[1].plot(curvatures[1,mask_up], regress_1.intercept + regress_1.slope * curvatures[1,mask_up], 
           'k:', label=f'Slope {regress_1.slope:.1e}')
#ax[1].plot(curvatures[1,mask_up], regress_2.intercept + regress_2.slope * curvatures[1,mask_up], 
#           'k:', label=f'Slope {regress_2.slope:.1e}')

ax[0].set_xlabel("Cylinder height H [m]")
ax[1].set_xlabel("Central curvature $\\kappa_{max}$ [$m^{-1}$]")
for axi in ax:
    axi.set_ylabel("Engineering strain $\\langle \epsilon_{xx} \\rangle_{xy}$ [-]")
    axi.legend()
    #axi.set_yscale('log')
    #axi.set_xscale('log')

plt.show()

In [ ]:
# Plot the evolution of the std of true strain as a function of time and maximal curvature

spatialStdXY_trueStrain_corrected = trueStrainXX_corrected.std(axis=(0,1))
spatialStdX_avgY_trueStrain_corrected = trueStrainXX_corrected.std(axis=0).mean(axis=0)

mask_up = (times_true < time_to_H.compute_time_rising(Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax))

fig, ax = plt.subplots(figsize=(10,4), ncols=2, tight_layout=True)
ax[0].set_title("True strain std as function of elevation")
ax[1].set_title("True strain std as function of curvature")

# First plot: elevation
ax[0].plot(heights, spatialStdXY_trueStrain_corrected, label="$\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$")
ax[0].plot(heights, spatialStdX_avgY_trueStrain_corrected, label="$\\langle \\sigma_{x} \\left( \epsilon^t_{xx} \\right) \\rangle_y$")

# Second plot: curvature
ax[1].plot(curvatures[1,mask_up], spatialStdXY_trueStrain_corrected[mask_up], label="$\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$")
ax[1].plot(curvatures[1,mask_up], spatialStdX_avgY_trueStrain_corrected[mask_up], label="$\\langle \\sigma_{x} \\left( \epsilon^t_{xx} \\right) \\rangle_y$")

ax[0].set_xlabel("Cylinder height H [m]")
ax[1].set_xlabel("Central curvature $\\kappa_{max}$ [$m^{-1}$]")
for axi in ax:
    axi.set_ylabel("True strain std $\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$ [-]")
    axi.legend()

plt.show()

In [ ]:
# Plot the evolution of the std of engineering strain as a function of time and maximal curvature

spatialStdXY_engStrain_corrected = engStrainXX_corrected.std(axis=(0,1))
spatialStdX_avgY_engStrain_corrected = engStrainXX_corrected.std(axis=0).mean(axis=0)
mask_up = (times_true < time_to_H.compute_time_rising(Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax))

# Compute the linear regressions
kMax_bounds_1 = [1, 4]
kMax_bounds_2 = [10, 12]
mask_bounds_1 = (kMax_bounds_1[0] < curvatures[1]) & (curvatures[1] < kMax_bounds_1[1])
mask_bounds_2 = (kMax_bounds_2[0] < curvatures[1]) & (curvatures[1] < kMax_bounds_2[1])

regress_1 = linregress(np.log(curvatures[1,mask_up&mask_bounds_1]), np.log(spatialStdXY_engStrain_corrected[mask_up&mask_bounds_1]))
regress_2 = linregress(np.log(curvatures[1,mask_up&mask_bounds_2]), np.log(spatialStdXY_engStrain_corrected[mask_up&mask_bounds_2]))


fig, ax = plt.subplots(figsize=(10,4), ncols=2, tight_layout=True)
ax[0].set_title("True strain std as function of elevation")
ax[1].set_title("True strain std as function of curvature")

# First plot: elevation
ax[0].plot(heights[mask_up], spatialStdXY_trueStrain_corrected[mask_up], label="$\\sigma_{xy} \\left( \epsilon_{xx} \\right)$")
ax[0].plot(heights[mask_up], spatialStdX_avgY_trueStrain_corrected[mask_up], label="$\\langle \\sigma_{x} \\left( \epsilon_{xx} \\right) \\rangle_y$")

# Second plot: curvature
ax[1].plot(curvatures[1,mask_up], spatialStdXY_trueStrain_corrected[mask_up], label="$\\sigma_{xy} \\left( \epsilon_{xx} \\right)$")
ax[1].plot(curvatures[1,mask_up], spatialStdX_avgY_trueStrain_corrected[mask_up], label="$\\langle \\sigma_{x} \\left( \epsilon_{xx} \\right) \\rangle_y$")
ax[1].plot(curvatures[1,mask_up], np.exp(regress_1.intercept) * curvatures[1,mask_up]**regress_1.slope, 'k:', label=f"Power {regress_1.slope:.2f}")

ax[0].set_xlabel("Cylinder height H [m]")
ax[1].set_xlabel("Central curvature $\\kappa_{max}$ [$m^{-1}$]")
for axi in ax:
    axi.set_ylabel("Engineering strain std $\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$ [-]")
    axi.legend()
    axi.set_yscale('log')
    axi.set_xscale('log')

plt.show()

In [ ]:
# Plot the evolution of the std of strain as a function of time and maximal curvature

range_trueStrain_corrected = trueStrainXX_corrected.max(axis=(0,1)) - trueStrainXX_corrected.min(axis=(0,1))
mask_wrongKappa = (heights < 0.002)

fig, ax = plt.subplots(figsize=(10,4), ncols=2, tight_layout=True)
ax[0].set_title("True strain std as function of elevation")
ax[1].set_title("True strain std as function of curvature")

# First plot: elevation
ax[0].plot(heights, range_trueStrain_corrected, label="$\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$")

# Second plot: curvature
ax[1].plot(curvatures[1,~mask_wrongKappa], range_trueStrain_corrected[~mask_wrongKappa], label="$\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$")

ax[0].set_xlabel("Cylinder height H [m]")
ax[1].set_xlabel("Central curvature $\\kappa_{max}$ [$m^{-1}$]")
for axi in ax:
    axi.set_ylabel("True strain std $\\sigma_{xy} \\left( \epsilon^t_{xx} \\right)$ [-]")
    axi.legend()

plt.show()

## Fourier analysis

In [ ]:
# Compute the FFT

# Remove mean strain for each frame
trueStrain_noMean = np.empty_like(trueStrainXX_corrected)
for frame_index in range(trueStrain_noMean.shape[-1]):
    trueStrain_noMean[:,:,frame_index] = trueStrainXX_corrected[:,:,frame_index] - trueStrainXX_corrected[:,:,frame_index].mean()

# Get coordinates and frequencies
x0_arr = coords[0,0,:,0,0]
dx = x0_arr[1] - x0_arr[0]
x_freq = fft.fftfreq(x0_arr.size, dx)
mask_posFreq = (x_freq > 0.)

# Compute the FFT
trueStrain_FFT = np.abs(fft.fft(trueStrain_noMean, axis=0))**2
trueStrain_avgFFT = trueStrain_FFT.mean(axis=1)

trueStrain_avgFFT.shape

In [ ]:
# At a given frame

frame_index = -1
y_elem_list = np.arange(0,25,5)

y0_arr = coords[0,1,0,:,0]

frame_true = frames_true[frame_index]
time_true = times_true[frame_index]
height = heights[frame_index]

print(f'Time: {time_true}s')

# Get strain FFT strain
trueStrain_FFT_frame = trueStrain_FFT[:,:,frame_index]
trueStrain_avgFFT_frame = trueStrain_avgFFT[:,frame_index]

# Plot the FFT
fig, ax = plt.subplots(tight_layout=True)

ax.plot(x_freq[mask_posFreq], trueStrain_avgFFT_frame[mask_posFreq], c='k', label="y-averaged")
for y_elem in y_elem_list:
    ax.plot(x_freq[mask_posFreq], trueStrain_FFT_frame[mask_posFreq, y_elem], label=f"y_pix={y0_arr[y_elem]:.0f}")

ax.legend()

ax.set_title("Fourier transform of the true strain")
ax.set_xlabel('Spatial frequency [cycle/pix]')
ax.set_ylabel('True strain spectral energy $ | \hat{\epsilon^t} |^2$ [-]')

plt.show()

In [ ]:
# Animation of the FFT evolution

nbFrames = trueStrain_FFT.shape[-1]
frameIndex_start = -1

fig, ax = plt.subplots(figsize=(6,5), tight_layout=True)
fig.set_dpi(100)

# Set labels
ax.set_title(f"Fourier transform of the true strain [H = {1e3 * heights[frameIndex_start]:.1f}mm]")
ax.set_xlabel('Spatial frequency [cycle/pix]')
ax.set_ylabel('True strain spectral energy $ | \hat{\epsilon^t} |^2$ [-]')

# Set bounds
x0Max = x0.max()
fftMax = trueStrain_FFT.max()
ax.set_xbound([-0.05*x0Max, 1.05*x0Max])
ax.set_ybound([-0.05*fftMax, 1.05*fftMax])

# Plot starting frame
plot, = ax.plot(x_freq[mask_posFreq], trueStrain_avgFFT[mask_posFreq,frameIndex_start])

def update(frame_index):
    plot.set_ydata(trueStrain_avgFFT[mask_posFreq, frame_index])
    ax.set_title(f"Fourier transform of the true strain [H = {1e3 * heights[frame_index]:.1f}mm]")
    return plot,

anim_fft = animation.FuncAnimation(fig, update, frames=nbFrames, repeat=False, interval=100)


In [ ]:
# Animation to HTML

anim_html_fft = HTML(anim_fft.to_html5_video())
anim_html_fft

## Correlations

In [ ]:
def corr_y(strain, y1_elem, y2_elem):
    return np.sum((strain[:,y1_elem] - strain[:,y1_elem].mean(axis=0)) * (strain[:,y2_elem] - strain[:,y2_elem].mean(axis=0)), axis=0)

def corr_x(strain, x1_elem, x2_elem):
    return np.sum((strain[x1_elem,:] - strain[x1_elem,:].mean(axis=0)) * (strain[x2_elem,:] - strain[x2_elem,:].mean(axis=0)), axis=0)

In [ ]:
# Compute the correlation with the middle slices

nb_xElem, nb_yElem, nb_frames = trueStrainXX_corrected.shape
middle_y = nb_yElem // 2
middle_x = nb_xElem //2

x0_arr = coords[0,0,:,0,0]
y0_arr = coords[0,1,0,:,0]


# Compute correlations
correlationY_trueStrain = np.zeros((nb_yElem, nb_frames))
for y_elem in range(nb_yElem):
    correlationY_trueStrain[y_elem] = corr_y(trueStrainXX_corrected, middle_y, y_elem)

correlationX_trueStrain = np.zeros((nb_xElem, nb_frames))
for x_elem in range(nb_xElem):
    correlationX_trueStrain[x_elem] = corr_x(trueStrainXX_corrected, middle_x, x_elem)

# Normalize correlation
for frame_index in range(1,nb_frames):
    correlationY_trueStrain[:,frame_index] /= correlationY_trueStrain[middle_y,frame_index]
    correlationX_trueStrain[:,frame_index] /= correlationX_trueStrain[middle_x,frame_index]


In [ ]:
# Plot the x and y correlations

frame_index = 105
correlationY_trueStrain_frame = correlationY_trueStrain[:,frame_index]
correlationX_trueStrain_frame = correlationX_trueStrain[:,frame_index]

fig, ax = plt.subplots(figsize=(10,4), ncols=2, tight_layout=True)

ax[0].plot(y0_arr, correlationY_trueStrain_frame)
ax[1].plot(x0_arr, correlationX_trueStrain_frame)

ax[0].set_title("Vertical correlation with middle slice")
ax[1].set_title("Horizontal correlation with middle slice")

ax[0].set_xlabel("$Y_0$ position [pix]")
ax[1].set_xlabel("$X_0$ position [pix]")

for axi in ax:
    axi.axhline(0., ls=':', c='k')
    axi.axhline(1., ls=':', c='k')
    axi.set_ylabel("True strain correlation [-]")

plt.show()